In [17]:
import os
import pandas as pd
import numpy as np
import torch
from transformers import ElectraForSequenceClassification, ElectraConfig, AutoTokenizer, ElectraModel

from konlpy.tag import Mecab

In [10]:
MODEL_NAME = "monologg/koelectra-small-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
electra_config = ElectraConfig.from_pretrained(MODEL_NAME)
model = ElectraModel.from_pretrained(MODEL_NAME)

In [36]:
data_df = pd.read_csv("../data/train/train.tsv", sep="\t", header=None)
data_df

,0,1,2,3,4,5,6,7,8
0,wikipedia-24896-25-30-33-19-21,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,30,33,자동차,19,21,단체:제작
1,wikipedia-12728-224-5-7-42-44,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,5,7,27석,42,44,관계_없음
2,wikipedia-28460-3-0-7-9-12,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,0,7,UEFA,9,12,단체:별칭
3,wikipedia-11479-37-24-26-3-5,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,24,26,공격수,3,5,인물:직업/직함
4,wikipedia-15581-6-0-2-32-40,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,0,2,퍼쿤 씨 인트라팃,32,40,인물:부모님
...,...,...,...,...,...,...,...,...,...
8995,wikipedia-5414-12-15-21-0-4,2002년 FIFA 월드컵 사우디아라비아와의 1차전에서 독일은 8-0으로 승리하였는...,사우디아라비아,15,21,2002년,0,4,관계_없음
8996,wikipedia-10384-4-12-14-0-1,일본의 2대 메이커인 토요타와 닛산은 시장 점유율을 높이기 위한 신차 개발을 계속하...,토요타,12,14,일본,0,1,단체:본사_국가
8997,wikipedia-25913-6-8-10-93-106,방호의의 손자 방덕룡(方德龍)은 1588년(선조 21년) 무과에 급제하고 낙안군수로...,방덕룡,8,10,선무원종공신(宣武原從功臣),93,106,인물:직업/직함
8998,wikitree-12062-15-0-3-46-47,LG전자는 올해 초 국내시장에 출시한 2020년형 ‘LG 그램’ 시리즈를 이달부터 ...,LG전자,0,3,북미,46,47,관계_없음


In [63]:
data_idx = 2
sentence = data_df.iloc[data_idx, 1]
e1_name = data_df.iloc[data_idx, 2]
e2_name = data_df.iloc[data_idx, 5]
print(sentence)
print(e1_name)
print(e2_name)

유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 2017년 대회부터 UEFA U-21 축구 선수권 대회 참가국을 8개국에서 12개국으로 확대하기로 결정했다.
유럽 축구 연맹
UEFA


In [64]:
print(tokenizer.tokenize(sentence))
print(tokenizer.tokenize(e1_name))
print(tokenizer.tokenize(e2_name))

['유럽', '축구', '연맹', '(', 'UEFA', ')', '집행', '##위원회', '##는', '2014', '##년', '1', '##월', '24', '##일', '##에', '열린', '회의', '##를', '통해', '2017', '##년', '대회', '##부터', 'UEFA', 'U', '-', '21', '축구', '선수', '##권', '대회', '참가', '##국', '##을', '8', '##개', '##국', '##에', '##서', '12', '##개', '##국', '##으로', '확대', '##하', '##기', '##로', '결정', '##했', '##다', '.']
['유럽', '축구', '연맹']
['UEFA']


In [65]:
print(tokenizer.encode(sentence))
print(tokenizer.encode(e1_name))
print(tokenizer.encode(e2_name))

[2, 6755, 7325, 9929, 12, 22038, 13, 7757, 13666, 4034, 7043, 4556, 21, 4501, 6592, 4366, 4073, 6867, 6356, 4110, 6260, 7651, 4556, 6622, 6406, 22038, 57, 17, 6591, 7325, 6463, 4046, 6622, 7137, 4113, 4292, 28, 4217, 4113, 4073, 4129, 6300, 4217, 4113, 10749, 6626, 4279, 4031, 4239, 6393, 4398, 4176, 18, 3]
[2, 6755, 7325, 9929, 3]
[2, 22038, 3]


In [66]:
e1_encoded_idx = [tokenizer.encode(sentence).index(tokenizer.encode(e1_name)[idx]) for idx in range(1, len(tokenizer.encode(e1_name)) - 1)]
e2_encoded_idx = [tokenizer.encode(sentence).index(tokenizer.encode(e2_name)[idx]) for idx in range(1, len(tokenizer.encode(e2_name)) - 1)]
print(e1_encoded_idx)
print(e2_encoded_idx)

[1, 2, 3]
[5]
